In [4]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 10 10:54:41 2015

@author: DimitrisHerrera
@adaptador: Miguel Andrade
"""
import numpy as np
from scipy import stats

#==============================================================================
# The Self Calibrating Palmer Drought Severity Index Code
#==============================================================================
#=============================================================================
# scPDSI.py - Emergent Climate Risk Laboratory (ECRL), Cornell University 
# October 2016
#=============================================================================
"""
This program calculates the "self-calibrating" Palmer Drought Severity Index for a given station. It integrates functions to calculate the Potential
evapotranspiration using the FAO Penman-Monteith or the Thornthwaite method. However, PET data can be also added to the computation of the index without
calculating it internally. Data inputs must be in inches per month. The function has two options to calculate duration factors: (1) using only 
consecutive values with the same sign, or using consecutive values regardless of the sign.
"""
#==============================================================================
# Water Balance Function   
#==============================================================================
#This function computes the water balance of the PDSI. 
def WaterBalance(pre, pe, awc, ss, newss, su, newsu, r, l, et, ro):
    '''
    This function computes the water balance used by PDSI. Input variables, e.g., 
    precipitation, evaporanspiration and available water capaciy, should be in 
    inches.
    
    The variable names are:
    pre -- (float) precipitation (inches/month)
    pe -- (float) potential evapotranspiration (inches/month)
    awc -- (float) available water capacity (inches)  
    ss -- (float) previous month surface (upper) soil layer of PDSI, which has a fixed thickness of one inch.
    su -- (float) previous month lower soil layer of PDSI. Thickness varies based on the available water
    capacity data used.
    newss -- (float) current month surface soil layer of PDSI. 
    newsu -- (float) current month lower soil layer of PDSI. 
    r -- (float) soil water recharge.
    l -- (float) soil water loss.
    et -- (float) evapotranspiration
    ro -- (float) runoff.
    
    The current time-step value for each output variable depends on their values
    in the previous time-step (i.e., the previous month)''' 
    
    dif = pe - pre 
    awctop = 1.0
    awcbot = awc - awctop
        
    if pe > pre:
        rs = 0.
        ru = 0.
        r = rs + ru        
        
        if ss > (dif):
            sl = dif
            ul = 0.
            newss = ss - sl
            newsu = su
            
        else:
            sl = ss
            ul = ((dif) - sl) * (su/awc) 
            ul = min(ul, su)
            newss = 0.
            newsu = su - ul
        
        l = sl + ul
        et = pre + l
        ro = 0.
        
    else:# // Now pre > pe begins. 
        sl = 0.            
        ul = 0.
        l = sl + ul
        et = pe
        
        if (-1*dif) > (awctop - ss):
            rs = awctop - ss
            newss = 1.
            
            if (pre - pe - rs) < (awcbot - su):
                ru = (-1*dif) - rs
                ro = 0.
                
            else:
                ru = awcbot - su
                ro = pre - pe - rs - ru
                
            newsu = su + ru
            r = rs + ru
            
        else:
            newss = (ss) + (-1*dif)
            newsu = su
            ro = 0.0    
            r = -1*dif
              
    ss = newss
    su = newsu        
        
    return  ss, su, r, l, et, ro

#==============================================================================
# Potentials:

#==============================================================================
def potentials(pe, pre, ss, su, awc):
    '''
    This function calculates the potential values and water balance coefficients 
    needed to compute the moisture anomaly index (Z) of PDSI
    The variables names used are:
    pr --  potential recharge.
    pro --  potential runoff.
    pl -- potential loss
    This functions needs outputs from the water balance function'''
    
    pr = awc - (ss + su) # potential recharge 
    
    if pre > awc:
        pro = pre - pr #potential runoff
    else:
        pro = awc - pr
    
    
    if ss >= pe:
        pl = pe #potential loss
    else:
        pl = ((pe - ss) * su) / (awc) + ss
        pl = min(pl,pro)
        

    return pr, pro, pl
#==============================================================================
# Z index Function calculates the moisture anomaly index of the PDSI.
#==============================================================================
    
def Z_index(pre, pe, et, pr, r, pro, ro, pl, l, cal_index_beg, cal_index_end):
    '''
    This function computes PDSI's moisture anomaly index "Z"; "phat", which is 
    the expected precipitation to maintain moisture balance; "d", which is the 
    difference between the actual and the expected precipitaiton. It is the first
    approximation of the moisture anomaly index; "alpha, beta, gama, and delta"
    coefficients 
    '''
    
    '''
    If the calibration period is different from the number of
    months, use this: Z_index(pre, pe, et, pr, r, pro, ro, pl, l, i, j)
    where i is the initial calibration period and j the end. Then,
    pre = pre[i:j]
    pe = pe[i:j]
    et = et[i:j]... and so on.'''

    m = 12  
    ETsum, PEsum, Rsum, PRsum, ROsum, PROsum, Lsum, \
    PLsum, PREsum, D, alpha, beta, gamma, delta, T = \
    (np.zeros(12) for _ in range(15)) 
    '''
    cal_index_beg is the index of the first month of the calibration period.
    cal_index_end is the last month of the calibration period.
    If cal_index_end = 0, then the entire time series will be used as the 
    calibration period.'''
    
    if cal_index_end == 0:
        pre_cal = pre
        pe_cal = pe
        et_cal = et
        pr_cal = pr
        r_cal = r
        pro_cal = pro
        ro_cal = ro
        pl_cal = pl
        l_cal = l
      
    else:
        pre_cal = pre[cal_index_beg : cal_index_end]
        pe_cal = pe[cal_index_beg : cal_index_end]
        et_cal = et[cal_index_beg : cal_index_end]
        pr_cal = pr[cal_index_beg : cal_index_end]
        r_cal = r[cal_index_beg : cal_index_end]
        pro_cal = pro[cal_index_beg : cal_index_end]
        ro_cal = ro[cal_index_beg : cal_index_end]
        pl_cal = pl[cal_index_beg : cal_index_end]
        l_cal = l[cal_index_beg : cal_index_end]
    
    
    for i in range(m):
        PREsum[i] = sum(pre_cal[i::m])
        PEsum[i] = sum(pe_cal[i::m])
        ETsum[i] = sum(et_cal[i::m])
        PRsum[i] = sum(pr_cal[i::m])
        Rsum[i] = sum(r_cal[i::m])
        PROsum[i] = sum(pro_cal[i::m])
        ROsum[i] = sum(ro_cal[i::m])
        PLsum[i] = sum(pl_cal[i::m])
        Lsum[i] = sum(l_cal[i::m])
        '''
        This step calculates the PDSI's "climatocally appropriate for 
        existing condictions (CAFEC) climate variables: CAFEC-evapotranspiration, 
        CAFEC-recharge, CAFEC-runoff, and CAFEC-loss.
        '''
        
        if(PEsum[i] != 0.0):
            alpha[i] = ETsum[i] / PEsum[i]
        else:
            if(ETsum[i] == 0.0):
                alpha[i] = 1.0
            else:
                alpha[i] = 0.0
        
        
        if(PRsum[i] != 0.0):
            beta[i] = Rsum[i] / PRsum[i]
        else:
            if(Rsum[i] == 0.0):
                beta[i] = 1.0
            else:
                beta[i] = 0.0
            
            
        if(PROsum[i] != 0.0):
            gamma[i] = ROsum[i] / PROsum[i]
        else:
            if(ROsum[i] == 0.0):
                gamma[i] = 1.0
            else:
                gamma[i] = 0.0  
            
            
        if(PLsum[i] != 0.0):
            delta[i] = Lsum[i] / PLsum[i]
        else:
            delta[i] = 0.0
    
    # Tile the coefficients to match with the period length.
    alpha = np.tile(alpha, int((len(pe))/12))    
    beta = np.tile(beta, int((len(pe))/12))
    gamma = np.tile(gamma, int((len(pe))/12))    
    delta = np.tile(delta, int((len(pe))/12))
    
    '''
    With the CAFEC calculated, the CAFEC-precipitation (Phat) is computed, which is 
    like the "expected" or "potential" precipitation based on a location's
    climatology
    '''
    Phat = (alpha * pe) + (beta * pr) + (gamma * pro)\
         - (delta * pl)
    
    '''
    Once Phat is calculated, the moisture departure (d) is computed as d = pre - Phat
    '''    
    d = pre - Phat
    
    if cal_index_end != 0:   
        dabs = abs(d[cal_index_beg : cal_index_end])
    else:
        dabs = abs(d)    
    
    for i in range(m):       
        D[i] = sum(dabs[i::m])
               
    Dbar = D / int((len(pe_cal))/12)
    
    '''
    From D, the first approximations of the climatic characteristic (k) and the
    moisture anomaly index (z) are calculated.
    '''
    
    T = (PEsum + Rsum + ROsum) / (PREsum + Lsum)
    T = np.where((PREsum + Lsum)==0,0.,T) # This vaiable is needed to calculate
                                          # the climatic characteristic.
        
    k = (1.5) * np.log10((T + 2.8) / Dbar) + 0.5  #1st Climatic Characteristic
    k = np.where(Dbar == 0., 0.5, k)   
    k = np.tile(k, int((len(pe))/12))
    
    '''
    First approximation of the moisture anomaly index.
    '''
    z = d * k 
           
    '''
    From this function, the outputs are: first approximations of the moisture 
    anomaly index (z) and the climatic characteristic (k), the precipitation 
    deficit (d), and characteristic'''
    
    return z, k, d, Phat

#==============================================================================
# DURATION FACTORS
#==============================================================================
#==============================================================================
# Sum_Z function
#==============================================================================
def sum_Z(data, length):
          
    sum_z = np.array([np.sum(data[length*i:length*i+length])\
    for i in range(int(len(data)/length))])

    return sum_z
#==============================================================================
# LeastSquares function to find the slope and y-intercept using a specific 
# correlation threshold.
#==============================================================================
def LeastSquares(x, y, c_tol):
   
    n = len(x)
    correlation = np.corrcoef(x,y)[1,0]
    slope, inter, r_value, p_value, std_err = stats.linregress(x, y)
    
    i = n - 1
    while ((abs(correlation)) < c_tol and i > 3):
       this_x = x[:-1]
       this_y = y[:-1]

       sumX = sum(this_x)
       sumY = sum(this_y)
       sumX2 = sum(this_x**2)
       sumY2 = sum(this_y**2)
       sumXY = sum(this_x * this_y)

       SSX = sumX2 - (sumX * sumX)/i
       SSY = sumY2 - (sumY * sumY)/i
       SSXY = sumXY - (sumX * sumY)/i    

       correlation = SSXY / (np.sqrt(SSX) * np.sqrt(SSY))

       x = this_x
       y = this_y
       n = len(x)
       i = n - 1
       slope = SSXY / SSX

    if slope > 0:
        y_inter = y - slope * x
        j = np.argmax(y_inter)
        intercept = y_inter[j]
        
    else:
        y_inter = y - slope * x
        j = np.argmin(y_inter)
        intercept = y_inter[j]
         
    
    return slope, intercept

#==============================================================================
# DurFact function    
#==============================================================================
'''
This function computes the duration factors, which represent the linear 
relationship between the length and severity of droughts/pluvials depending 
on the local climate conditions. The output variables are the slope and
y-intercept for dry (md & bd, respectively) and wet (mw & bw, respectively) 
intervals.
'''

def DurFact(z, dur_mode, dur_cal_range):

   
    if dur_cal_range == 'all':
        length1 = np.arange(3,15,3)
        length2 = np.arange(18,54,6)
        length = np.concatenate((length1, length2))
    
    else:
        length = np.arange(3,18,3)
        
    dry_spells = np.zeros(len(length))
    wet_spells = np.zeros(len(length))
    
    z_neg = np.where(z>0,0.,z)
    z_pos = np.where(z<0,0.,z)
    
    '''
    If mode == 'same' the summation of z is done by only adding values of the 
    same sign. Otherwise the summation takes into account all the consecutives 
    z values.
    '''
    for i in range(len(length)):
        
        if dur_mode=='same':
            try:
                dry_spells[i] = np.min(sum_Z(z_neg,length[i]))
                wet_spells[i] = np.max(sum_Z(z_pos,length[i]))
            
            except ValueError:
                pass
    
        else:
            dry_spells[i] = np.min(sum_Z(z,length[i]))
            wet_spells[i] = np.max(sum_Z(z,length[i]))
    
    md, bd = LeastSquares(length, dry_spells, 0.85)
    
    md = md / -4
    bd = bd / -4

    mw, bw = LeastSquares(length,wet_spells, 0.85)
        
    mw = mw / 4
    bw = bw / 4
    
    return md, bd, mw, bw
        
    
#==============================================================================
# COMPUTING AND SELECTING X 
#==============================================================================

#==============================================================================
# Function Clear X3
#==============================================================================
'''
This function puts X3=0 where Pe != 0 nor 100%, which means that 
there is not a dry (wet) interval established.
'''
def ClearX3(x3, pro_copy):

    x = np.zeros_like(x3)

    for i in range(len(x3)):
        if pro_copy[i] == 0 or pro_copy[i] == 100:
            if abs(x3[i]) != 0:
                x[i] = x3[i]

        else:
            x[i] = 0.0
    return x
    
#==============================================================================
#Function supx
#==============================================================================
'''
Thisfunction selects the final PDSI based on the probability of ending-beginning 
a dry/wet interval. The details of this are described in Palmer (1965) and Wells 
et al. (2004)
'''
def supx(x, x1,x2):

    for i in range(len(x1)-2,-1,-1):
        if x[i] == 0.0:
            if x[i+1] == x1[i+1] or x[i+1] > 0.0:
                x[i] = x1[i]

            elif x[i+1] == x2[i+1] or x[i+1] < 0.0:
                x[i] = x2[i]
    
    return x

#==============================================================================
def SelectX(x, x1, x2):

    I = np.ma.where(np.logical_and(x == 0.0, x1 == 0.0))
    J = np.ma.where(np.logical_and(x == 0.0, x2 == 0.0))
    K = np.ma.where(np.logical_and((np.logical_and(x == 0.0, x1 != 0.0)),x2 != 0.0))
    x[J] = x1[J]
    x[I] = x2[I]
    x[K] = 0.0
    if x[len(x)-1] == 0.0:
        if abs(x1[len(x1)-1]) > abs(x2[len(x2)-1]):
            x[len(x)-1] == x1[len(x1)-1]
            x2[len(x2)-1] = 0.0
        else:
            x[len(x)-1] = x2[len(x2)-1]
            x1[len(x1)-1] = 0.0

    x = supx(x,x1,x2)

    return x
#==============================================================================
# CLEAR PRO FUNCTION: this function puts to zero "island" of probabilities that 
# never reach 100% (i.e., probabilities != 100 surrounded by zeros).
# This function replaces the backstrapping function in the original PDSI.
#==============================================================================  
def Clear_pro(pro):
    
    for i in range(len(pro)):
        if pro[i] == 0:
            if pro[i-1] < 100:
                pro[i-1] = 0.
                
    return pro
    
#==============================================================================
# fill_Xs
#==============================================================================
def Fill_Xs(x1, x2, x3):

    if (x1 > 1.) and (x3 == 0.):
        x3 = x1
        x1 = 0.
      
    elif (x2 < -1.) and (x3 == 0.): # matlab code x2<= 1
        x3 = x2
        x2 = 0.
        
    return x1, x2, x3   

#==============================================================================
# Between zeros
#==============================================================================
def between0s(x1, x2, x3, z, nz, c, pro, v):
    v = 0.
    pro = 0.
    x1 = 0.
    x2 = 0.
    newx3 = (c*x3) + (z/nz) #ADDED
    x3 = newx3  # ADDED
    
    return x1, x2, x3, pro, v 
        
#==============================================================================
# UD AND UW FUNCTION: 
# this function calculates the variables required to calculate the 
# probability of ending an established interval.
#==============================================================================
def func_ud_wd(z, q, v, x1, x2, x3, pro, uw, ud, ze, mw, md, bw, bd, mode):
    
    if x3 >= 0:
        Wd = 1
        m = mw
        b = bw
  
    else:
        Wd = -1
        m = md
        b = bd
        
    c = 1 - (m / (m + b))    
    nz = (m+b)      
            
    if mode=='ud':
        
        ud = z - 0.15
        newv = ud + min(v, 0.)
        if newv >= 0.:
            q = 0.                                  
            x1, x2, newx3, newpro, newv = between0s(x1, x2, x3, z, nz, c, pro, v)
        
        ze = (m+b)*(Wd*0.5 - c*x3)
        
        if pro == 100:
            q = ze
            
        else:
            q = ze + v

        newpro = (newv / q) * 100
        if newpro > 100:
            newpro = 100.
            newx3 = 0.
        else:
            newx3 = (c*x3) + (z/nz)

        x3 = newx3
        pro = newpro
        v = newv
        x1, x2, x3 = Fill_Xs(x1, x2, x3)
                
    else:

        uw = z + 0.15
        newv = uw + max(v, 0.)
        if newv <= 0.:
            q = 0.
            x1, x2, newx3, newpro, newv = between0s(x1, x2, x3, z, nz, c, pro, v)
        
        ze = (m+b)*(Wd*0.5 - c*x3)
        
        if pro == 100:
            q = ze
        
        else:
            q = ze + v

        newpro = (newv / q) * 100
        if newpro > 100:
            newpro = 100.
            newx3 = 0.
            
        else:
            newx3 = (c*x3) + (z/nz)

        x3 = newx3
        pro = newpro
        v = newv
        x1, x2, x3 = Fill_Xs(x1, x2, x3)

    return q, v, x1, x2, x3, pro, uw, ud, ze
            
    

#==============================================================================
# HERE BEGINS THE COMPUTATION OF X, X1, X2 AND X3
#==============================================================================    
#==============================================================================
# COMPUTATION OF Xs
#==============================================================================

def CalcX(x1, x2, x3, z, md, bd, mw, bw):
        
    if x3 >= 0.:
        m = mw
        b = bw
    
    else:
        m = md
        b = bd
        
    c = 1 - (m / (m+b))
    nz = (m+b)  
    
    dryc = 1 - (md / (md+bd))
    dz = (md+bd)  
    
    wetc = 1 - (mw / (mw+bw)) 
    wz = (mw+bw)  
    
    newx1 = max(((wetc * x1) + (z/wz)), 0.)        
    newx2 = min(((dryc * x2) + (z/dz)), 0.)

    if x3 >= 0.:
        newx3 = (wetc * x3) + (z/wz)
        
    else:
        newx3 = (dryc * x3) + (z/dz)
    
    return newx1, newx2, newx3, c, dryc, wetc, nz, dz, wz

#==============================================================================
# PDSI CORE FUNCTION
#==============================================================================
def CalcPDSI(z, md, bd, mw, bw):
    x, x1, x2, x3, pro, ud, uw, ze, v, q, bt = \
    (np.zeros_like(z) for _ in range(11))
            
    for i in range(len(z)):

        x1[i], x2[i], x3[i], c, dryc, wetc, nz, dz, wz = CalcX(x1[i-1], x2[i-1], x3[i-1], z[i], md, bd, mw, bw)
                                                              
        if pro[i-1] == 100. or pro[i-1] == 0.:# No abatement underway, since a wet or dry spell will end
                                       #  if -0.5 =< x3 =< 0.5
            if x3[i-1] > -0.5 and x3[i-1] < 0.5:# > -0.5 and x3[i-1] < 0.5: # or, abs(x3) > 0.5 A dry or wet spell ends.
              
                v[i] = 0.
                pro[i] = 0.
                x3[i] = 0.
                x1[i], x2[i], x3[i] = Fill_Xs(x1[i], x2[i], x3[i])             

            elif x3[i-1] > 0.5:    
                if z[i] > 0.15:
                    x1[i], x2[i], x3[i], pro[i], v[i] = between0s(x1[i], x2[i], x3[i-1], z[i], nz, c, pro[i-1], v[i])
                                   
                else:
                    
                    q[i], v[i], x1[i], x2[i], x3[i], pro[i], uw[i], ud[i], ze[i] = \
                    func_ud_wd(z[i], q[i], v[i-1], x1[i], x2[i], x3[i-1], pro[i-1], uw[i], ud[i], ze[i], \
                    mw, md, bw, bd, 'ud')    
                
            elif x3[i-1] < -0.5:
                
                if z[i] < -0.15:
                    
                    x1[i], x2[i], x3[i], pro[i], v[i] = between0s(x1[i], x2[i], x3[i-1], z[i], nz, c, pro[i-1], v[i])

                else:
                                        
                    q[i], v[i], x1[i], x2[i], x3[i], pro[i], uw[i], ud[i], ze[i] = \
                    func_ud_wd(z[i], q[i], v[i-1], x1[i], x2[i], x3[i-1], pro[i-1], uw[i], ud[i], ze[i], \
                    mw, md, bw, bd, 'uw') 

        else:
            
            if x3[i-1] > 0.:

                q[i], v[i], x1[i], x2[i], x3[i], pro[i], uw[i], ud[i], ze[i] = \
                func_ud_wd(z[i], q[i], v[i-1], x1[i], x2[i], x3[i-1], pro[i-1], uw[i], ud[i], ze[i], \
                mw, md, bw, bd, 'ud') 

                              
            if x3[i-1] < 0.:
       
               q[i], v[i], x1[i], x2[i], x3[i], pro[i], uw[i], ud[i], ze[i] = \
               func_ud_wd(z[i], q[i], v[i-1], x1[i], x2[i], x3[i-1], pro[i-1], uw[i], ud[i], ze[i], \
               mw, md, bw, bd, 'uw') 
                    
    return x1, x2, x3, ud, uw, v, ze, q, pro

#==============================================================================
# scPDSI FUNCTION
#==============================================================================
def sc_PDSI(pre, pe, awc, cal_index_beg, cal_index_end, dur_mode, dur_cal_range): 
    
    maxpre = np.zeros((12))
    for i in range(12):
        maxpre[i] = np.max(pre[i::12])
    
    maxpre = np.tile(maxpre,int(len(pre)/12))
    
    SS, SU, L, ET, RS, RU, R, RO, SL, UL, PL, PR, PRO, K = \
    (np.zeros(len(pe)) for _ in range(14))
    
    for i in range(len(pre)):
        SS[0] = 1.0
        SU[0] = awc-SS[0]
        SS[i], SU[i], R[i], L[i], ET[i], RO[i] = \
        WaterBalance(pre[i], pe[i], awc, SS[i-1], SS[i], SU[i-1], SU[i], R[i], L[i], ET[i], RO[i])
        PR[i], PRO[i], PL[i] = potentials(pe[i], maxpre[i], SS[i-1], SU[i-1], awc)
        
    z, k, d, Phat = Z_index(pre, pe, ET, PR, R, PRO, RO, PL, L, cal_index_beg, cal_index_end)     
    
    if cal_index_end == 0:
        md, bd, mw, bw = DurFact(z, dur_mode, dur_cal_range)
    
    else:
        z_cal = z[cal_index_beg:cal_index_end]
        md, bd, mw, bw = DurFact(z_cal, dur_mode, dur_cal_range)
    
    x1, x2, x3, ud, uw, v, ze, q, pro = CalcPDSI(z, md, bd, mw, bw)
    
    pro_copy = np.copy(pro)
    for i in range(len(pro)):
        if pro_copy[i] == 0.0 and pro_copy[i-1] != 0.0:
            pro_copy = Clear_pro(pro_copy)
        
        
    x3_mod = ClearX3(x3,pro_copy)    
    
    #x = np.ma.copy(x3_mod)
    
    x = SelectX(x3_mod,x1,x2)
    
    if cal_index_end == 0:
        x_cal = x
    else:
        x_cal = x[cal_index_beg:cal_index_end]
    
    high = np.percentile(x_cal, 98)
    low = np.percentile(x_cal, 2)
            
    K = np.where(d<0, k*(-4/low), k*(4/high))
             
    Z = d * K #Second approximation of moisture anomaly (z)

    X1, X2, X3, Ud, Uw, V, Ze, Q, Pro = CalcPDSI(Z, md, bd, mw, bw)
        
    Pro_copy = np.copy(Pro)
    for i in range(len(Pro)):
        if Pro_copy[i] == 0.0 and Pro_copy[i-1] != 0.0:
            Pro_copy = Clear_pro(Pro_copy)
    
    X3_mod = ClearX3(X3,Pro_copy) 
    
    #X = np.ma.copy(X3_mod)
    
    X = SelectX(X3_mod,X1,X2)
    
    return X, X3_mod, Z


# =============================================================================
# This function calculates scPDSI using PET daily rates and monthly precipitation
# data.    
# =============================================================================

def pdsi_wrapper(pre, pe, awc, cal_index_beg, cal_index_end, dur_mode, dur_cal_range, years, data_rate_day, data_unit):
    '''
    =========================================================================================
    Main input variables:
    pre             :   (float) monthly precipitation data in mm/month. Precipitation 
                        data should have at least 30 years.
    pe              :   (float) monthly daily rates of potential evapotranspiration (PET) 
                        data in mm/month. PET data should have at least 30 years.
    awc             :   (float) available water capacity in inches.
    cal_index_beg   :   (float) index of the first month of the calibration period.
    cal_index_end   :   (float) tlast month of the calibration period.
    dur_mode        :   (string) Select the method to calculate the duration factors.
                        'same' calculates the duration factors using only negative
                        or positive PDSI values, else takes batch of both 
                        positive and negative PDSI months to calculate the duration
                        factors.
    dur_cal_range   :   (string) Select the range to calculate the linear regression
                        used to calculate the duration factors. 'part' only select
                        data in batches of 3 months from , while 'all' in batches of 
                        6 months. 
        
                        
    years           :   (int) The number of years the time series spans.
    data_rate_day   :   Input data is in mm/day and not in mm/month.
    
    index of the first month of the calibration period.
    cal_index_end is the last month of the calibration period.
    =========================================================================================
    
    "M" is a list of number of days per month, required to convert climate data from mm/day to 
    monthly totals. M[2::48] puts February to have 29 days for leap years , as long as the first
    year of the input time series is a leap year. Otherwise, the index of the first February in
    leap years should be changed accordingly.
    
    '''
    M = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    M = np.tile(M,years)
    M[2::48] = 29
    
    
    '''
    pre = pre*86400: convertion of precipitation and potential evapotranspiration from kg/m2/s 
    to mm (pre*86400) and then to inches (pre*0.0393700787). 
    Caveat: this is only required if precipitation data is in kg/m2/s as in most global climate 
    outputs (e.g., CMIPs outputs).
    '''
    if data_unit == "model":
        pre = pre*86400
        pe = pe*86400
    else:
        pre = pre
        pe = pe
    
    if data_rate_day == "true":
        pre = pre*M
        pe = pe*M
    else:
        pre = pre
        pe = pe*M
        
    P = pre*0.0393700787
    PE = pe*0.0393700787
    
    scpdsi, X3, Z = sc_PDSI(P, PE, awc, cal_index_beg, cal_index_end, dur_mode, dur_cal_range)
    return scpdsi
    
#Caribbean lat/lon indices: [:,200:550,150::]   
    
#It should be like this: (pdsi_wrapper(pre[12::,300,0],pet[12::,300,0],awc[300,0],0,600,"same","part",69))
# For 3D observational data, it should be called as:
#for i in range(340):
#    for j in range(280):
#        pdsi[:,i,j] = pdsi_wrapper(P[:,i,j], PET[:,i,j],AWC[i,j],0,0,"same","all",24,"false","data")
def aggregate_to_annual(data, years):
    """Agrupa datos mensuales en bloques anuales."""
    return np.array([np.sum(data[i * 12:(i + 1) * 12]) for i in range(years)])

def pdsi_wrapper_annual(pre, pe, awc, cal_index_beg, cal_index_end, dur_mode, dur_cal_range, years, data_rate_day, data_unit):
    """
    Wrapper modificado para calcular el scPDSI en base anual.
    """
    M = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    M = np.tile(M, years)
    M[2::48] = 29

    if data_unit == "model":
        pre = pre * 86400
        pe = pe * 86400

    if data_rate_day == "true":
        pre = pre * M
        pe = pe * M

    P = pre * 0.0393700787
    PE = pe * 0.0393700787

    # Agrupar los datos mensuales en bloques anuales
    P_annual = aggregate_to_annual(P, years)
    PE_annual = aggregate_to_annual(PE, years)

    # Calcular scPDSI anual
    scpdsi, X3, Z = sc_PDSI(P_annual, PE_annual, awc, cal_index_beg, cal_index_end, dur_mode, dur_cal_range)
    return scpdsi



In [6]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
# Cargar el dataset ERA5
ERA5 = xr.open_dataset('./ERA5_PTEUV_1980_2024.nc')

# Seleccionar las variables de precipitación (pre) y evapotranspiración potencial (pe)
pre = ERA5['tp']  # Total precipitation (mm/month)
pe = ERA5['pev']  # Potential evapotranspiration (mm/month)

# Convertir a matrices de NumPy para facilitar el bucle
NPpre = pre.values
NPpe = pe.values
print("Shape de PE:", NPpe.shape)
print("Shape de PRE:", NPpre.shape)

# Dimensiones del dataset
n_time, n_lat, n_lon = NPpre.shape

# Asegurarse de que el número de meses sea un múltiplo de 12 para tener años completos
n_years = n_time // 12
n_months = n_years * 12
NPpre = NPpre[:n_months, :, :]
NPpe = NPpe[:n_months, :, :]

# Definir parámetros para el cálculo de scPDSI
awc = 1.0  # Capacidad de agua disponible en pulgadas
cal_index_beg = 0
cal_index_end = 0  # Usar toda la serie para calibración
dur_mode = 'same'
dur_cal_range = 'all'
years = n_years  # Número de años (considerando datos mensuales)
data_rate_day = 'false'
data_unit = 'data'

# Crear una matriz vacía para almacenar los resultados del scPDSI
xxx = np.full((n_months, n_lat, n_lon), np.nan)  # Lleno con NaN por defecto

# Crear la matriz M con el número de días de cada mes, replicada para cada año completo
M = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
M = np.tile(M, n_years)

# Calcular el scPDSI para cada punto (lat, lon)
for eachI in tqdm(range(n_lat)):
    for eachJ in range(n_lon):
        pre_point = NPpre[:, eachI, eachJ]
        pe_point = NPpe[:, eachI, eachJ]

        # Verificar si hay datos válidos (evitar todo el punto con NaN)
        if not np.isnan(pre_point).all() and not np.isnan(pe_point).all():
            try:
                # Aplicar la función pdsi_wrapper solo si hay datos válidos
                scpdsi = pdsi_wrapper(pre_point, pe_point, awc, cal_index_beg, cal_index_end, dur_mode, dur_cal_range, years, data_rate_day, data_unit)
                xxx[:, eachI, eachJ] = scpdsi
            except ValueError as e:
                print(f"Error en el punto (lat: {eachI}, lon: {eachJ}): {e}")

# Convertir la matriz resultante a un DataArray de xarray
scpdsi_da = xr.DataArray(
    xxx,
    dims=["time", "lat", "lon"],
    coords={"time": pre.coords["time"][:n_months], "lat": pre.coords["lat"], "lon": pre.coords["lon"]},
    name="scPDSI"
)

# Añadir atributos al DataArray
scpdsi_da.attrs['long_name'] = 'Self-calibrating Palmer Drought Severity Index'
scpdsi_da.attrs['units'] = 'dimensionless'

# Visualizar un ejemplo del scPDSI en un momento específico
scpdsi_da.isel(time=0).plot(cmap='RdBu', figsize=(10, 6))
plt.title('scPDSI - Primer mes del dataset')
plt.show()


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'argo', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html